You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [2]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [3]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [4]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [5]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [6]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [7]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [8]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [10]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [11]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [12]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer:
Title: The Future of Artificial Intelligence: Latest Trends, Key Players, and Noteworthy News

Introduction:
Artificial Intelligence (AI) is rapidly transforming various industries, from healthcare to finance, and even entertainment. In this blog article, we will delve into the latest trends, key players, and noteworthy news in the world of AI.

Target Audience:
Our target audience includes tech enthusiasts, business professionals, and individuals curious about the advancements in AI. They are interested in le

I now can give a great answer

Final Answer: 

# The Future of Artificial Intelligence: Latest Trends, Key Players, and Noteworthy News

Artificial Intelligence (AI) has rapidly become a driving force in various industries worldwide, reshaping the way we live and work. Dating back to the 1950s, AI involves the simulation of human intelligence processes by machines, encompassing learning, reasoning, and self-correction. Over the years, AI has undergone significant evolution, leading to groundbreaking technological advancements.

In the realm of AI, one of the most notable trends is the swift progress of machine learning algorithms. These algorithms empower machines to enhance their performance over time by learning from data without explicit programming. Additionally, natural language processing (NLP) has seen substantial innovation, enabling machines to comprehend, interpret, and generate human language. AI's impact extends to sectors such as healthcare, finance, and transportation, st

- Display the results of your execution as markdown in the notebook.

In [13]:
from IPython.display import Markdown
Markdown(result)

# The Future of Artificial Intelligence: Latest Trends, Key Players, and Noteworthy News

Artificial Intelligence (AI) has rapidly become a driving force in various industries worldwide, reshaping the way we live and work. Dating back to the 1950s, AI involves the simulation of human intelligence processes by machines, encompassing learning, reasoning, and self-correction. Over the years, AI has undergone significant evolution, leading to groundbreaking technological advancements.

In the realm of AI, one of the most notable trends is the swift progress of machine learning algorithms. These algorithms empower machines to enhance their performance over time by learning from data without explicit programming. Additionally, natural language processing (NLP) has seen substantial innovation, enabling machines to comprehend, interpret, and generate human language. AI's impact extends to sectors such as healthcare, finance, and transportation, streamlining processes, boosting efficiency, and refining decision-making.

In the AI industry, tech giants like Google, Amazon, IBM, and Microsoft stand at the forefront, investing heavily in AI research and development. These companies have introduced cutting-edge technologies and solutions that revolutionize human-machine interactions. For instance, Google has introduced AI-powered applications such as Google Assistant and Google Translate, while Amazon has integrated AI into its e-commerce platform for personalized recommendations.

Recent breakthroughs in AI research have expanded the capabilities of machines, enabling advancements like self-driving cars and advanced robotics. While these developments bring new possibilities in automation and autonomy, they also raise ethical concerns regarding job displacement and data privacy. Policymakers, industry leaders, and researchers must address these concerns to ensure responsible development and deployment of AI technologies.

As AI continues to progress and shape our future, staying informed about the latest trends and developments in the field is crucial. By following reputable sources like Forbes, MIT Technology Review, and TechCrunch, readers can gain valuable insights into the potential impact of AI on society. The audience is encouraged to engage responsibly with AI technologies and share their perspectives to initiate discussions on the ethical and social implications of AI.

In conclusion, Artificial Intelligence is poised to revolutionize industries and foster innovation in the years to come. By staying abreast of the latest trends, key players, and noteworthy news in the AI industry, individuals can equip themselves for the forthcoming opportunities and challenges. Embracing the future of AI with curiosity and awareness is key to navigating the evolving landscape of technology.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [14]:
topic = "Custom Model Deplyment Bedrock"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Custom Model Deplyment Bedrock.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Title: Unveiling Custom Model Deployment Bedrock: A Comprehensive Guide

Introduction:
- Brief overview of Custom Model Deployment Bedrock
- Importance and benefits of utilizing Custom Model Deployment Bedrock

Key Points:
1. Latest Trends in Custom Model Deployment Bedrock
- Real-time monitoring and optimization capabilities
- Integration with various machine learning frameworks
- Customization options for specific business needs

2. Key Players in Custom Model Deployment Bedrock
- Leading companies 

I now can give a great answer
Final Answer: 

# Unveiling Custom Model Deployment Bedrock: A Comprehensive Guide

In the rapidly evolving landscape of artificial intelligence and machine learning, custom model deployment bedrock has emerged as a crucial tool for data scientists, machine learning engineers, and AI enthusiasts. This innovative technology offers real-time monitoring and optimization capabilities, integration with various machine learning frameworks, and customization options tailored to specific business needs. By leveraging custom model deployment bedrock, businesses can enhance their machine learning models and streamline their deployment processes effectively.

One of the latest trends in custom model deployment bedrock is the emphasis on real-time monitoring and optimization. This feature allows users to track the performance of their models continuously and make necessary adjustments in real-time. Additionally, the integration with various machine learning frameworks

In [15]:
Markdown(result)

# Unveiling Custom Model Deployment Bedrock: A Comprehensive Guide

In the rapidly evolving landscape of artificial intelligence and machine learning, custom model deployment bedrock has emerged as a crucial tool for data scientists, machine learning engineers, and AI enthusiasts. This innovative technology offers real-time monitoring and optimization capabilities, integration with various machine learning frameworks, and customization options tailored to specific business needs. By leveraging custom model deployment bedrock, businesses can enhance their machine learning models and streamline their deployment processes effectively.

One of the latest trends in custom model deployment bedrock is the emphasis on real-time monitoring and optimization. This feature allows users to track the performance of their models continuously and make necessary adjustments in real-time. Additionally, the integration with various machine learning frameworks provides flexibility and scalability, enabling seamless deployment across different platforms. Customization options further enhance the adaptability of custom model deployment bedrock to meet specific business requirements, making it a versatile solution for model deployment.

Leading companies in the AI and machine learning space are at the forefront of providing custom model deployment bedrock solutions. Through case studies showcasing successful deployments, these key players demonstrate the effectiveness and efficiency of custom model deployment bedrock in real-world scenarios. By partnering with industry leaders, businesses can access cutting-edge technology and expertise to optimize their model deployment processes and drive innovation in their organizations.

Recent advancements in deployment techniques have further improved the capabilities of custom model deployment bedrock. Success stories from industries utilizing this technology highlight the transformative impact it has on businesses, driving growth and competitive advantage. By staying informed on the latest news and updates in custom model deployment bedrock, decision-makers can make informed decisions and implement best practices to maximize the benefits of this innovative solution.

The benefits of utilizing custom model deployment bedrock are manifold. From enhanced performance and scalability to improved customization and flexibility, this technology offers a comprehensive solution for businesses looking to optimize their model deployment processes. By leveraging custom model deployment bedrock, organizations can achieve greater efficiency, accuracy, and innovation in their machine learning models, driving success and growth in an increasingly competitive market.

Case studies and success stories provide valuable insights into the practical applications of custom model deployment bedrock. By showcasing real-world examples of successful deployments, businesses can learn from industry leaders and implement best practices in their own organizations. These case studies serve as inspiration for decision-makers looking to enhance their model deployment processes and achieve tangible results through the adoption of custom model deployment bedrock.

Implementing custom model deployment bedrock effectively requires a strategic approach and a deep understanding of the technology. By following best practices and leveraging expert insights on model deployment, businesses can successfully integrate custom model deployment bedrock into their existing workflows. Through careful planning, testing, and optimization, organizations can maximize the benefits of this technology and drive innovation in their machine learning models.

For those interested in learning more about custom model deployment bedrock, resources such as industry reports, whitepapers, case studies, and expert interviews offer valuable information and insights. By exploring these resources, readers can gain a deeper understanding of the technology, its applications, and best practices for implementation. By staying informed and continually learning about custom model deployment bedrock, individuals and organizations can stay ahead in the fast-evolving field of AI and machine learning.

In conclusion, custom model deployment bedrock is a game-changing technology that offers immense potential for businesses looking to enhance their machine learning models and optimize their deployment processes. By staying informed on the latest trends, key players, and noteworthy news in the industry, decision-makers can make informed decisions and drive innovation in their organizations. With the right resources and a strategic approach to implementation, custom model deployment bedrock can revolutionize the way businesses approach model deployment and achieve success in the competitive landscape of AI and machine learning.

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).